In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DBLP Analysis") \
    .getOrCreate()

# Load the dataset
df = spark.read.json("./dblp-ref/*.json", multiLine=True)

# Show the schema to understand the structure
df.printSchema()

# Display a sample of the data
df.show(5)

# Count the number of records
print(f"Number of records: {df.count()}")


root
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- year: long (nullable = true)

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|            abstract|             authors|                  id|n_citation|          references|               title|               venue|year|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|Based on biologic...|[Guoping Pang, La...|4aa69add-3978-480...|         8|[04754a28-6bf4-4d...|Dynamic analysis ...|Mathematics and C...|2008|
|In this paper, a ...

In [2]:
# Display summary statistics
df.describe().show()

# Check for missing values (excluding `isnan`)
from pyspark.sql import functions as F
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Distribution of citations
df.select("n_citation").describe().show()

# Check for null abstracts and titles
df.filter(df.abstract.isNull() | df.title.isNull()).show()


+-------+--------------------+--------------------+-----------------+--------------------+--------------------+------------------+
|summary|            abstract|                  id|       n_citation|               title|               venue|              year|
+-------+--------------------+--------------------+-----------------+--------------------+--------------------+------------------+
|  count|                   4|                   4|                4|                   4|                   4|                 4|
|   mean|                NULL|                NULL|             14.5|                NULL|                NULL|           2011.25|
| stddev|                NULL|                NULL|23.96525262402492|                NULL|                NULL|3.9475730941089733|
|    min|AdaBoost algorith...|00127ee2-cb05-48c...|                0|A Heterogeneous S...|Mathematics and C...|              2008|
|    max|The purpose of th...|4ab3735c-80f1-472...|               50|Preliminary De

In [3]:
!pip install langid
import langid
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Function to detect language using langid
def detect_language(text):
    if text is None:
        return None
    lang, _ = langid.classify(text)
    return lang

# Registering UDF
lang_detect_udf = udf(detect_language, StringType())

# Add a new column for language detection
df = df.withColumn("language", lang_detect_udf(df.abstract))

# Filter only English documents
df = df.filter(df.language == 'en')


  Using cached langid-1.1.6-py3-none-any.whl


In [4]:
from pyspark.sql.functions import col, lower, regexp_replace, split
from pyspark.ml.feature import StopWordsRemover

# Custom stop words
custom_stop_words = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 
                     'author', 'figure', 'rights', 'reserved', 'permission', 'used', 'using', 
                     'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 
                     'CZI', 'www']

# Lowercase and remove punctuation
df_cleaned = df.withColumn("cleaned_abstract", lower(col("abstract")))
df_cleaned = df_cleaned.withColumn("cleaned_abstract", regexp_replace(col("cleaned_abstract"), r'[!()\-\[\]{};:\'",<>./?@#$%^&*_~]', ''))

# Tokenize the text
df_tokenized = df_cleaned.withColumn("tokenized_abstract", split(col("cleaned_abstract"), " "))

# Remove stop words
remover = StopWordsRemover(inputCol="tokenized_abstract", outputCol="filtered_abstract", 
                           stopWords=StopWordsRemover().getStopWords() + custom_stop_words)
df_filtered = remover.transform(df_tokenized)

# Show the cleaned dataframe
df_filtered.select("abstract", "cleaned_abstract", "tokenized_abstract", "filtered_abstract").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
from pyspark.ml.feature import HashingTF, IDF

# Apply TF
hashingTF = HashingTF(inputCol="filtered_abstract", outputCol="rawFeatures", numFeatures=20000)
df_featurized = hashingTF.transform(df_filtered)

# Apply IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(df_featurized)
df_vectorized = idf_model.transform(df_featurized)

# Select only the columns we need
df_vectorized = df_vectorized.select("id", "title", "features")
df_vectorized.show(5)


+--------------------+--------------------+--------------------+
|                  id|               title|            features|
+--------------------+--------------------+--------------------+
|4aa69add-3978-480...|Dynamic analysis ...|(20000,[28,42,274...|
|4ab3735c-80f1-472...|A new approach of...|(20000,[78,274,46...|
|00127ee2-cb05-48c...|Preliminary Desig...|(20000,[1072,1241...|
|001eef4f-1d00-4ae...|A Heterogeneous S...|(20000,[193,274,2...|
+--------------------+--------------------+--------------------+



In [6]:
# !pip install langid
# from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql.types import StringType
# from pyspark.ml.feature import HashingTF, IDF, PCA
# from pyspark.ml.clustering import KMeans
# import matplotlib.pyplot as plt
# import langid

# # Initialize Spark session with maxResultSize configuration
# spark = SparkSession.builder \
#     .appName("PCA_KMeans") \
#     .config("spark.driver.maxResultSize", "4g") \
#     .getOrCreate()

# # Load the dataset
# df = spark.read.json("./dblp-ref/*.json", multiLine=True)

# # Apply TF-IDF to the filtered abstract
# hashingTF = HashingTF(inputCol="filtered_abstract", outputCol="rawFeatures", numFeatures=20000)
# df_featurized = hashingTF.transform(df_filtered)

# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idf_model = idf.fit(df_featurized)
# df_vectorized = idf_model.transform(df_featurized)

# # Select only the columns we need
# df_vectorized = df_vectorized.select("id", "title", "features")
# df_vectorized.show(5)

# # Sample the dataframe for PCA and KMeans due to large size
# def sample_data(df, fraction, max_attempts=5):
#     attempt = 0
#     sampled_df = df.sample(fraction=fraction, seed=42)
#     while sampled_df.isEmpty() and attempt < max_attempts:
#         fraction *= 2  # Increase fraction to get more data
#         sampled_df = df.sample(fraction=fraction, seed=42)
#         attempt += 1
#     if sampled_df.isEmpty():
#         raise ValueError("Sampled DataFrame is empty after several attempts.")
#     return sampled_df

# # Sample the data with initial fraction 0.1
# df_sampled = sample_data(df_vectorized, 0.1)

# # Verify that the sampled DataFrame is not too large
# print(f"Sampled DataFrame count: {df_sampled.count()}")

# # Apply PCA to reduce dimensions (e.g., to 20 components)
# pca = PCA(k=20, inputCol="features", outputCol="pca_features")
# pca_model = pca.fit(df_sampled)
# df_pca = pca_model.transform(df_sampled)

# # Determine the optimal number of clusters using the elbow method
# costs = []
# for k in range(2, 21):
#     kmeans = KMeans(k=k, seed=1, featuresCol="pca_features")
#     model = kmeans.fit(df_pca)
#     cost = model.summary.trainingCost
#     costs.append(cost)

# # Plot the elbow curve
# plt.figure(figsize=(10, 6))
# plt.plot(range(2, 21), costs, marker='o')
# plt.xlabel('Number of Clusters')
# plt.ylabel('Cost')
# plt.title('Elbow Method For Optimal k')
# plt.show()


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import HashingTF, IDF, PCA
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt

# Initialize Spark session with maxResultSize configuration
spark = SparkSession.builder \
    .appName("PCA_KMeans") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

# Load the dataset
df = spark.read.json("./dblp-ref/*.json", multiLine=True)

# Apply TF-IDF to the filtered abstract
hashingTF = HashingTF(inputCol="filtered_abstract", outputCol="rawFeatures", numFeatures=20000)
df_featurized = hashingTF.transform(df_filtered)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(df_featurized)
df_vectorized = idf_model.transform(df_featurized)

# Select only the columns we need
df_vectorized = df_vectorized.select("id", "title", "features")
df_vectorized.show(5)

# Sample the dataframe for PCA and KMeans due to large size
def sample_data(df, fraction, max_attempts=5):
    attempt = 0
    sampled_df = df.sample(fraction=fraction, seed=42)
    while sampled_df.count() == 0 and attempt < max_attempts:
        fraction *= 2  # Increase fraction to get more data
        sampled_df = df.sample(fraction=fraction, seed=42)
        attempt += 1
    if sampled_df.count() == 0:
        raise ValueError("Sampled DataFrame is empty after several attempts.")
    return sampled_df

# Sample the data with initial fraction 0.1
df_sampled = sample_data(df_vectorized, 0.1)

# Verify that the sampled DataFrame is not too large
print(f"Sampled DataFrame count: {df_sampled.count()}")
df_sampled.show(5)

# Apply PCA to reduce dimensions (e.g., to 20 components)
pca = PCA(k=20, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_sampled)
df_pca = pca_model.transform(df_sampled)

# Determine the optimal number of clusters using the elbow method
costs = []
for k in range(2, 21):
    kmeans = KMeans(k=k, seed=1, featuresCol="pca_features")
    model = kmeans.fit(df_pca)
    cost = model.summary.trainingCost
    costs.append(cost)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(2, 21), costs, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()


+--------------------+--------------------+--------------------+
|                  id|               title|            features|
+--------------------+--------------------+--------------------+
|4aa69add-3978-480...|Dynamic analysis ...|(20000,[28,42,274...|
|4ab3735c-80f1-472...|A new approach of...|(20000,[78,274,46...|
|00127ee2-cb05-48c...|Preliminary Desig...|(20000,[1072,1241...|
|001eef4f-1d00-4ae...|A Heterogeneous S...|(20000,[193,274,2...|
+--------------------+--------------------+--------------------+

Sampled DataFrame count: 3
+--------------------+--------------------+--------------------+
|                  id|               title|            features|
+--------------------+--------------------+--------------------+
|4aa69add-3978-480...|Dynamic analysis ...|(20000,[28,42,274...|
|00127ee2-cb05-48c...|Preliminary Desig...|(20000,[1072,1241...|
|001eef4f-1d00-4ae...|A Heterogeneous S...|(20000,[193,274,2...|
+--------------------+--------------------+-------------------

IllegalArgumentException: requirement failed: Cannot aggregate object of size 1600080000 Bytes, as it's bigger than maxResultSize (1073741824 Bytes)

In [8]:
# Assume the optimal number of clusters is determined to be 10
optimal_k = 10

# Train the K-means model
kmeans = KMeans(k=optimal_k, seed=1, featuresCol="pca_features")
model = kmeans.fit(df_pca)

# Make predictions
df_clusters = model.transform(df_pca)
df_clusters.select("id", "title", "prediction").show(5)


NameError: name 'df_pca' is not defined

In [9]:
from pyspark.ml.linalg import Vectors
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Function to calculate cosine similarity
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return float(dot_product / (norm_v1 * norm_v2))

cosine_similarity_udf = udf(cosine_similarity, DoubleType())

# Build search engine function
def recommend_papers(title, top_n=5):
    # Find the cluster of the given paper title
    paper_cluster = df_clusters.filter(df_clusters.title == title).select("prediction").collect()[0][0]
    
    # Get all papers in the same cluster
    cluster_papers = df_clusters.filter(df_clusters.prediction == paper_cluster)
    
    # Get the features of the given paper
    paper_features = df_clusters.filter(df_clusters.title == title).select("features").collect()[0][0]
    
    # Calculate cosine similarity and recommend top N papers
    cluster_papers = cluster_papers.withColumn("similarity", cosine_similarity_udf(cluster_papers.features, Vectors.dense(paper_features.toArray())))
    recommendations = cluster_papers.orderBy("similarity", ascending=False).limit(top_n)
    
    return recommendations.select("title", "similarity").collect()

# Example usage
recommended_papers = recommend_papers("A new approach of....", top_n=5)
for rec in recommended_papers:
    print(f"Title: {rec['title']}, Similarity: {rec['similarity']}")


NameError: name 'df_clusters' is not defined

In [ ]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile('dblp.v10.zip', 'r') as zip_ref:
    zip_ref.extractall('dblp_data')

# List the contents of the directory to verify
print(os.listdir('dblp_data'))


In [ ]:
import shutil

# Directory containing the original dataset
original_dir = 'dblp_data'
# Directory to store copies
copies_dir = 'dblp_data_copies'

# Create the directory if it doesn't exist
os.makedirs(copies_dir, exist_ok=True)

# Create 1000 copies
for i in range(1000):
    dest_dir = os.path.join(copies_dir, f'dblp_copy_{i}')
    shutil.copytree(original_dir, dest_dir)
    
print(f"Created {len(os.listdir(copies_dir))} copies of the dataset.")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF, PCA
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
import langid

# Initialize Spark session with the desired configuration
spark = SparkSession.builder \
    .appName("PCA_KMeans") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Load the dataset
df = spark.read.json("dblp_data/*.json", multiLine=True)

# Sample the dataframe for PCA and KMeans due to large size
def sample_data(df, fraction, max_attempts=5):
    attempt = 0
    sampled_df = df.sample(fraction=fraction, seed=42)
    while sampled_df.isEmpty() and attempt < max_attempts:
        fraction *= 2  # Increase fraction to get more data
        sampled_df = df.sample(fraction=fraction, seed=42)
        attempt += 1
    if sampled_df.isEmpty():
        raise ValueError("Sampled DataFrame is empty after several attempts.")
    return sampled_df

# Sample the data with initial fraction 0.1
df_sampled = sample_data(df, 0.1)

# Verify that the sampled DataFrame is not too large
print(f"Sampled DataFrame count: {df_sampled.count()}")

# Apply PCA to reduce dimensions (e.g., to 20 components)
pca = PCA(k=20, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_sampled)
df_pca = pca_model.transform(df_sampled)

# Determine the optimal number of clusters using the elbow method
costs = []
for k in range(2, 21):
    kmeans = KMeans(k=k, seed=1, featuresCol="pca_features")
    model = kmeans.fit(df_pca)
    cost = model.summary.trainingCost
    costs.append(cost)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(2, 21), costs, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()
